# **Application of classical algorithms for image segmentation**



**Data download**

In [ ]:
!git clone https://github.com/mwarylak/BraTS_Thesis.git

In [ ]:
!pip install kaggle
!kaggle datasets download -d awsaf49/brats2020-training-data
!unzip brats2020-training-data.zip

In [25]:
import numpy as np
import os
import h5py
import random
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import local_binary_pattern
from tempfile import TemporaryFile
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
from sklearn.multiclass import OneVsOneClassifier

**Loading data**




In [12]:
directory = "/content/BraTS2020_training_data/content/data"
h5_files = [f for f in os.listdir(directory) if f.endswith('.h5')]
model_saving = True

plt.style.use('ggplot')
plt.rcParams['figure.facecolor'] = '#FFFFFF'
plt.rcParams['text.color']       = '#474646'

Method to load single images

In [28]:
def sample_h5_file(dir, files, index, name = ""):
    if name != "":
      sample_file_path = os.path.join(dir, name)
    else:
      sample_file_path = os.path.join(dir, files[index])
    with h5py.File(sample_file_path, 'r') as file:
      image = file['image'][()].transpose(2, 0, 1)
      mask = file['mask'][()].transpose(2, 0, 1)

    print(sample_file_path)

    return image, mask

def test_sample(sample_file_path, index):
    with h5py.File(sample_file_path[index], 'r') as file:
      image = file['image'][()].transpose(2, 0, 1)
      mask = file['mask'][()].transpose(2, 0, 1)

    return image, mask

Methods for displaying MRI images

In [5]:
def display_image_channels(image, title='Image Channels'):
    channel_names = ['T1-weighted (T1)', 'T1-weighted post contrast (T1c)', 'T2-weighted (T2)', 'FLAIR']
    fig, axes = plt.subplots(2, 2, figsize=(8, 8))
    for idx, ax in enumerate(axes.flatten()):
        channel_image = image[idx, :, :]
        ax.imshow(channel_image, cmap='magma') #viridis
        ax.axis('off')
        ax.set_title(channel_names[idx])
    plt.tight_layout()
    plt.suptitle(title, fontsize=20, y=1.03)
    plt.show()

def display_mask_channels_as_rgb(mask, title='Mask Channels'):
    channel_names = ['Necrotic (NEC)', 'Edema (ED)', 'Tumour (ET)']
    fig, axes = plt.subplots(1, 3, figsize=(9.75, 5))
    for idx, ax in enumerate(axes):
        rgb_mask = np.zeros((mask.shape[1], mask.shape[2], 3), dtype=np.uint8)
        rgb_mask[..., idx] = mask[idx, :, :] * 255
        ax.imshow(rgb_mask)
        ax.axis('off')
        ax.set_title(channel_names[idx])
    plt.suptitle(title, fontsize=20, y=0.93)
    plt.tight_layout()
    plt.show()

def overlay_masks_on_image(image, mask, title='Brain MRI with Tumour Masks Overlay'):
    t1_image = image[0, :, :]
    t1_image_normalized = (t1_image - t1_image.min()) / (t1_image.max() - t1_image.min())

    rgb_image = np.stack([t1_image_normalized] * 3, axis=-1)
    color_mask = np.stack([mask[0, :, :], mask[1, :, :], mask[2, :, :]], axis=-1)
    rgb_image = np.where(color_mask, color_mask, rgb_image)

    plt.figure(figsize=(7, 7))
    plt.imshow(rgb_image)
    plt.title(title, fontsize=18, y=1.02)
    plt.axis('off')
    plt.show()

def display_combined_mask_as_rgb(mask, title='Combined Mask as RGB'):
    rgb_mask = np.zeros((mask.shape[1], mask.shape[2], 3), dtype=np.uint8)

    rgb_mask[..., 0] = mask[0, :, :] * 255
    rgb_mask[..., 1] = mask[1, :, :] * 255
    rgb_mask[..., 2] = mask[2, :, :] * 255

    plt.figure(figsize=(8, 8))
    plt.imshow(rgb_mask)
    plt.title(title, fontsize=20, y=1.02)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def display_combined_predicted_mask_as_rgb(pred_mask, title='Predicted Combined Mask'):
    rgb_pred_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)
    rgb_pred_mask[pred_mask == 1] = [255, 0, 0]
    rgb_pred_mask[pred_mask == 2] = [0, 255, 0]
    rgb_pred_mask[pred_mask == 3] = [0, 0, 255]

    plt.figure(figsize=(8, 8))
    plt.imshow(rgb_pred_mask)
    plt.title(title, fontsize=20, y=1.02)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def overlay_prediction(image, pred_mask, title='Prediction Overlay'):
    t1_image = image[0, :, :]
    t1_image_normalized = (t1_image - t1_image.min()) / (t1_image.max() - t1_image.min())

    rgb_image = np.stack([t1_image_normalized] * 3, axis=-1)

    rgb_pred = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.float32)
    rgb_pred[..., 0] = (pred_mask == 1).astype(np.float32)
    rgb_pred[..., 1] = (pred_mask == 2).astype(np.float32)
    rgb_pred[..., 2] = (pred_mask == 3).astype(np.float32)

    overlay = np.clip(rgb_image + rgb_pred * 0.5, 0, 1)

    return overlay

def overlay_masks(image, mask, title='Brain MRI with Tumour Masks Overlay'):
    t1_image = image[0, :, :]
    t1_image_normalized = (t1_image - t1_image.min()) / (t1_image.max() - t1_image.min())

    rgb_image = np.stack([t1_image_normalized] * 3, axis=-1)
    color_mask = np.stack([mask[0, :, :], mask[1, :, :], mask[2, :, :]], axis=-1)
    rgb_image = np.where(color_mask, color_mask, rgb_image)

    return rgb_image

def display_side_by_side(image, pred_mask, mask, title="Prediction Overlay"):
    overlay_pred = overlay_prediction(image, pred_mask)
    overlay_mask = overlay_masks(image, mask)

    plt.figure(figsize=(16, 8))

    plt.subplot(1, 2, 1)
    plt.imshow(overlay_pred)
    plt.title(title, fontsize=18, y=1.02)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(overlay_mask)
    plt.title('Brain MRI with Tumour Masks Overlay', fontsize=18, y=1.02)
    plt.axis('off')

    plt.show()

**Removal of incomplete MRI scans**

In [ ]:
h5_files_cleaned = []
for i in range(len(h5_files)):
  _, mask = sample_h5_file(directory, h5_files, i)
  for channel in range(mask.shape[0]):
      if mask[channel].any():
          h5_files_cleaned.append(h5_files[i])
          break

with open("complete_MRI.txt", "w") as file:
    for element in h5_files_cleaned:
        file.write(element + "\n")

**Load only full MRII scans**

In [8]:
with open("/content/BraTS_Thesis/Files/h5files.txt", "r") as file:
    files = [line.strip() for line in file]

**Sample counts**

In [ ]:
def count_classes_in_masks(directory, h5_files):
    total_non_zero_counts = [0, 0, 0]
    samples = {0: [], 1: [], 2: []}

    for i in range(len(h5_files)):
        image, mask = sample_h5_file(directory, h5_files, i)

        for channel_index in range(mask.shape[0]):
            non_zero_count = np.count_nonzero(mask[channel_index])
            total_non_zero_counts[channel_index] += non_zero_count

            if channel_index in samples:
                samples[channel_index].extend(np.where(mask[channel_index] > 0)[0].tolist())  # Indeksy niezerowe

    for channel_index in range(len(total_non_zero_counts)):
        print(f'Kanał {channel_index}: {total_non_zero_counts[channel_index]} wartości różne od zera')#, {total_zero_counts[channel_index]} wartości równe zeru.')

count_classes_in_masks(directory, h5_files)

#**Segmentation**


In [10]:
def extract_features(image):
    features = []
    for channel in range(image.shape[0]):
        channel_image = image[channel, :, :]
        lbp = local_binary_pattern(channel_image, P=8, R=1.0, method='uniform')
        features.append(channel_image.flatten())
        features.append(lbp.flatten())
    return np.array(features).T

def train_model(model, images, directory, batch_size, norm_type = ''):
    print(f'Model: {model}\nNormalization: {norm_type}')
    feature_file = TemporaryFile()
    label_file = TemporaryFile()

    scaler = StandardScaler()

    files = images
    batch_size = batch_size

    for batch_start in range(0, len(files), batch_size):
        print(f'Batch: {batch_start}')
        batch_files = files[batch_start:batch_start + batch_size]
        batch_features = []
        batch_labels = []

        for file_name in batch_files:
            sample_file_path = os.path.join(directory, file_name)
            with h5py.File(sample_file_path, 'r') as file:
                image = file['image'][()].transpose(2, 0, 1)
                if 'mask' not in file:
                    print(f"No mask found for {file_name}, skipping.")
                    continue
                mask = file['mask'][()].transpose(2, 0, 1)

                if norm_type == 'min-max':
                  for i in range(image.shape[0]):
                    min_val = np.min(image[i])
                    image[i] = image[i] - min_val
                    max_val = np.max(image[i]) + 1e-4
                    image[i] = image[i] / max_val

                elif norm_type == 'z_score':
                  mean = image.mean()
                  std = image.std()
                  image = (image - mean) / std

                elif norm_type == 'percent':
                  p2, p98 = np.percentile(image, (2, 98))
                  image_clipped = np.clip(image, p2, p98)
                  image = (image_clipped - p2) / (p98 - p2)

                else:
                  pass

                nec_mask = mask[0, :, :]
                ed_mask = mask[1, :, :]
                et_mask = mask[2, :, :]

                multi_class_mask = np.zeros_like(nec_mask)
                multi_class_mask[nec_mask > 0] = 1
                multi_class_mask[ed_mask > 0] = 2
                multi_class_mask[et_mask > 0] = 3

                features = extract_features(image)
                labels = multi_class_mask.flatten()
                batch_features.append(features)
                batch_labels.append(labels)

        if batch_features:
            batch_features = np.vstack(batch_features)
            batch_labels = np.hstack(batch_labels)

            if batch_start == 0:
                scaler.fit(batch_features)
            batch_features_scaled = scaler.transform(batch_features)

            feature_file.seek(0, os.SEEK_END)
            np.save(feature_file, batch_features_scaled)

            label_file.seek(0, os.SEEK_END)
            np.save(label_file, batch_labels)

    feature_file.seek(0)
    label_file.seek(0)

    try:
        all_features = np.load(feature_file, allow_pickle=True)
        all_labels = np.load(label_file, allow_pickle=True)
        print(f"Read {all_features.shape} features and {all_labels.shape} labels.")
    except Exception as e:
        print("There was a problem while loading the data:", e)

    print(f"Number of samples: {len(all_features)}, Number of labels: {len(all_labels)}")
    if len(all_features) == 0 or len(all_labels) == 0:
        raise ValueError("No data to split. Check if the images contain masks.")

    print("Unique classes in labels:", np.unique(all_labels))

    X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.2, random_state=42)

    multimodel = model
    multimodel.fit(X_train, y_train)

    y_pred = multimodel.predict(X_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    return y_pred, y_test, multimodel

def save_model(model, filename='model.joblib'):
  dump(model, filename)

def load_model(filename):
  return load(filename)

Test image to check the results after training

In [22]:
image, mask = sample_h5_file(directory, files, 105)

nec_mask = mask[0, :, :]
ed_mask = mask[1, :, :]
et_mask = mask[2, :, :]

multi_class_mask = np.zeros_like(nec_mask)
multi_class_mask[nec_mask > 0] = 1
multi_class_mask[ed_mask > 0] = 2
multi_class_mask[et_mask > 0] = 3

features = extract_features(image)
labels = multi_class_mask.flatten()

/content/BraTS2020_training_data/content/data/volume_324_slice_57.h5


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


# Support Vector Machine

**Training**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = OneVsOneClassifier(LinearSVC(random_state=42))
y_svm_pred, y_svm_test, svm_model = train_model(model, files, directory, batch_size=200, norm_type='min-max')

features_scaled = scaler.transform(features)

svm_pred_mask = svm_model.predict(features_scaled)
svm_pred_mask_image = svm_pred_mask.reshape(multi_class_mask.shape)

if model_saving:
  save_model(svm_model, '/content/BraTS_Thesis/models_weights/svm_model.joblib')

display_side_by_side(image, svm_pred_mask_image, mask)

**Testing**

In [ ]:
folder_path = '/content/BraTS_Thesis/Files/Test_Samples'
test_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

index = random.randint(0, 149)
image, mask = test_sample(test_files, index=index)

nec_mask = mask[0, :, :]
ed_mask = mask[1, :, :]
et_mask = mask[2, :, :]

multi_class_mask = np.zeros_like(nec_mask)
multi_class_mask[nec_mask > 0] = 1
multi_class_mask[ed_mask > 0] = 2
multi_class_mask[et_mask > 0] = 3

scaler = StandardScaler()
features = extract_features(image)
labels = multi_class_mask.flatten()

X_train, X_test, _, _ = train_test_split(features, labels, test_size=0.2, random_state=42)
_ = scaler.fit_transform(X_train)
_ = scaler.transform(X_test)
features_scaled = scaler.transform(features)

svm_model = load_model('/content/BraTS_Thesis/models_weights/svm_model.joblib')

svm_pred_mask = svm_model.predict(features_scaled)
svm_pred_mask_image = svm_pred_mask.reshape(multi_class_mask.shape)

display_side_by_side(image, svm_pred_mask_image, mask)

# Random Forest

**Training**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = RandomForestClassifier(n_estimators=100, random_state=42)
y_rf_pred, y_rf_test, rf_model = train_model(model, files, directory, batch_size=200, norm_type = '')

features_scaled = scaler.transform(features)

rf_pred_mask = rf_model.predict(features_scaled)
rf_pred_mask_image = rf_pred_mask.reshape(multi_class_mask.shape)

if model_saving:
  save_model(rf_model, '/content/BraTS_Thesis/models_weights/random_forest.joblib')

display_side_by_side(image, rf_pred_mask_image, mask)

**Testing**

In [ ]:
folder_path = '/content/BraTS_Thesis/Files/Test_Samples'
test_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

index = random.randint(0, 149)
image, mask = test_sample(test_files, index=index)

nec_mask = mask[0, :, :]
ed_mask = mask[1, :, :]
et_mask = mask[2, :, :]

multi_class_mask = np.zeros_like(nec_mask)
multi_class_mask[nec_mask > 0] = 1
multi_class_mask[ed_mask > 0] = 2
multi_class_mask[et_mask > 0] = 3

scaler = StandardScaler()
features = extract_features(image)
labels = multi_class_mask.flatten()

X_train, X_test, _, _ = train_test_split(features, labels, test_size=0.2, random_state=42)
_ = scaler.fit_transform(X_train)
_ = scaler.transform(X_test)
features_scaled = scaler.transform(features)

rf_model = load_model('/content/BraTS_Thesis/models_weights/random_forest.joblib')

rf_pred_mask = rf_model.predict(features_scaled)
rf_pred_mask_image = rf_pred_mask.reshape(multi_class_mask.shape)

display_side_by_side(image, rf_pred_mask_image, mask)